In [1]:
import numpy as np
import pandas as pd
import cv2 as cv
import pytesseract
from glob import glob
import spacy
import re
import string

In [2]:
def cleanText(txt):
    
    whitespace = string.whitespace
    punctuation = '!#$%&\'()*+:;<=>?[\\]^`{|}~'
    tableWhitespace = str.maketrans('','',whitespace)
    tablePunctuation = str.maketrans('','',punctuation)

    text = str(txt)
    text = text.lower()
    removeWhitespace = text.translate(tableWhitespace)
    removePunctuation = removeWhitespace.translate(tablePunctuation)
    
    return str(removePunctuation)

In [3]:
model_ner = spacy.load('./output/model-best/')

In [4]:
image = cv.imread('../images/card_00.jpg')

# cv.imshow('businesscard', image)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [5]:
tessData = pytesseract.image_to_data(image)

In [6]:
tessList = list(map(lambda x:x.split('\t'),tessData.split('\n')))
df = pd.DataFrame(tessList[1:],columns=tessList[0])
df.dropna(inplace=True) #Drop missing values
df['text'] = df['text'].apply(cleanText) #Apply cleanText function on text column

In [7]:
df_clean = df.query('text !="" ') #Ignore whitespace
content = " ".join([w for w in df_clean['text']]) #Join every word in text column concatenated by spaces

In [8]:
doc = model_ner(content)

In [9]:
from spacy import displacy
displacy.render(doc,style='ent')

In [10]:
dockJSON = doc.to_json()

doc_text = dockJSON['text']
# doc_text #Testing
df_tokens = pd.DataFrame(dockJSON['tokens']) #Create data frame from tokens
# df_tokens.head() #Testing
df_tokens['token'] = df_tokens[['start','end']].apply(lambda x:doc_text[x[0]:x[1]], axis=1) #Add text table
# df_tokens.head(10) #Testing
right_table = pd.DataFrame(dockJSON['ents'])[['start','label']] #Take the entities table
df_tokens = pd.merge(df_tokens,right_table,how='left',on='start') #And left-join it with the tokens+text table
df_tokens.fillna('O',inplace=True) #Replace all NaN fields
# df_tokens.head(10) #Testing

In [11]:
df_clean.head(10)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
4,5,1,1,1,1,1,527,29,96,34,91,mike
5,5,1,1,1,1,2,645,29,225,34,69,polinowski
9,5,1,2,1,1,1,628,78,62,19,91,chief
10,5,1,2,1,1,2,701,78,171,19,90,procrastinator
12,5,1,2,1,2,1,701,102,174,13,38,i
30,5,1,6,1,1,1,21,299,53,16,91,"39c,"
31,5,1,6,1,1,2,83,300,53,13,92,street
32,5,1,6,1,1,3,143,300,36,15,93,"318,"
33,5,1,6,1,1,4,188,300,65,17,91,boeung
34,5,1,6,1,1,5,261,300,42,17,92,keng


In [12]:
df_clean['end'] = df_clean['text'].apply(lambda x: len(x) + 1).cumsum() - 1
df_clean['start'] = df_clean[['text','end']].apply(lambda x: x[1] - len(x[0]),axis=1)

/tmp/ipykernel_26186/3321973009.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['end'] = df_clean['text'].apply(lambda x: len(x) + 1).cumsum() - 1
/tmp/ipykernel_26186/3321973009.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['start'] = df_clean[['text','end']].apply(lambda x: x[1] - len(x[0]),axis=1)


In [13]:
df_card = pd.merge(df_clean,df_tokens[['start','token','label']],how='inner',on='start')
df_card.head(10)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
0,5,1,1,1,1,1,527,29,96,34,91,mike,4,0,mike,B-NAME
1,5,1,1,1,1,2,645,29,225,34,69,polinowski,15,5,polinowski,I-NAME
2,5,1,2,1,1,1,628,78,62,19,91,chief,21,16,chief,O
3,5,1,2,1,1,2,701,78,171,19,90,procrastinator,36,22,procrastinator,O
4,5,1,2,1,2,1,701,102,174,13,38,i,38,37,i,O
5,5,1,6,1,1,1,21,299,53,16,91,"39c,",43,39,39c,O
6,5,1,6,1,1,2,83,300,53,13,92,street,50,44,street,O
7,5,1,6,1,1,3,143,300,36,15,93,"318,",55,51,318,O
8,5,1,6,1,1,4,188,300,65,17,91,boeung,62,56,boeung,O
9,5,1,6,1,1,5,261,300,42,17,92,keng,67,63,keng,O


In [14]:
bb_df = df_card.query("label ! = 'O'")

In [15]:
img = image.copy() #first take a copy of the original image

for x,y,w,h,label in bb_df[['left','top','width','height','label']].values: #write data into an array
  x = int(x)
  y = int(y)
  w = int(w)
  h = int(h)

  cv.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2) #Draw rectangle around entities
  cv.putText(img,str(label),(x,y),cv.FONT_HERSHEY_PLAIN,1,(255,0,0),1) #Add a tag from value of label

cv.imshow('Prediction', img)
cv.waitKey(10000)
cv.destroyAllWindows()

In [16]:
bb_df['label'] = bb_df['label'].apply(lambda x: x[2:])
bb_df.head()

/tmp/ipykernel_26186/2854881843.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['label'] = bb_df['label'].apply(lambda x: x[2:])


,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
0,5,1,1,1,1,1,527,29,96,34,91,mike,4,0,mike,NAME
1,5,1,1,1,1,2,645,29,225,34,69,polinowski,15,5,polinowski,NAME
15,5,1,7,1,1,2,52,366,45,13,92,855,101,98,855,PHONE
16,5,1,7,1,1,3,105,365,23,16,93,0,103,102,0,PHONE
17,5,1,7,1,1,4,136,366,20,13,96,23,106,104,23,PHONE


In [17]:
class groupgen():
    def __init__(self):
        self.id = 0
        self.text = ''
        
    def getgroup(self,text):
        if self.text == text: #If entity has the same label - group them under the same id
            return self.id
        else:
            self.id +=1 #Else increment
            self.text = text
            return self.id
        
grp_gen = groupgen()

In [18]:
bb_df['group'] = bb_df['label'].apply(grp_gen.getgroup)

/tmp/ipykernel_26186/472836701.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['group'] = bb_df['label'].apply(grp_gen.getgroup)


In [19]:
bb_df[['top','left','width','height']] = bb_df[['top','left','width','height']].astype(int)
bb_df['right'] = bb_df['left'] +  bb_df['width'] #Calculate right edge
bb_df['bottom'] = bb_df['top'] +  bb_df['height'] #Calculate bottom edge

/home/xiaodie/.local/lib/python3.9/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/tmp/ipykernel_26186/3758920108.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['right'] = bb_df['left'] +  bb_df['width'] #Calculate right edge
/tmp/ipykernel_26186/3758920108.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [20]:
col_group = ['top','bottom','left','right','label','token','group']
group_tag_img = bb_df[col_group].groupby(by='group')

In [21]:
img_tagging = group_tag_img.agg({
    'top':min,
    'bottom':max,
    'left':min,
    'right':max,
    'label':np.unique,
    'token':lambda x: " ".join(x) #Join all words together seperated by a space
})

In [26]:
img_tagging

,top,bottom,left,right,label,token
group,,,,,,
1,29,63,527,870,NAME,mike polinowski
2,365,423,21,241,PHONE,855 0 23 21 59 60 email
3,409,427,77,244,EMAIL,me@example.com
4,453,471,21,216,WEB,www.some-place.com


In [29]:
card_bb = image.copy()
for left,right,top,bottom,label,token in img_tagging.values:
    cv.rectangle(card_bb, (left,top), (right,bottom), (0,255,0), 2)
    cv.putText(card_bb,str(label),(left,top),cv.FONT_HERSHEY_PLAIN,1,(255,0,255),1)

cv.imshow('Name Card with Bounding Boxes', card_bb)
cv.waitKey(10000)
cv.destroyAllWindows()